In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_table("/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip")
test = pd.read_table("/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip")

In [ ]:
display(train, test)

In [ ]:
train["Phrase"][0]

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(all_data["Phrase"])

In [ ]:
all_text = tk.texts_to_sequences(all_data['Phrase'])

In [ ]:
# tk.word_counts["the"]
tk.word_index

In [ ]:
from keras.preprocessing.sequence import pad_sequences 
all_pad = pad_sequences(all_text) #길이 제한
all_pad.shape

In [ ]:
def load_embedding(file_path):
    embedding = {}
    with open(file_path) as f:
        for i in f:
            value = i.strip().split()
            word = value[0]
            vector = np.asarray(value[1:],dtype="float32") #ram소모를 줄일 수 있음
            embedding[word] = vector
    return embedding

In [ ]:
embedding = load_embedding("/kaggle/input/fasttext-crawl-300d-2m/crawl-300d-2M.vec")

In [ ]:
def filter_embedding(embedding,word_index,vocab_size,dim):
    embedding_matrix = np.zeros([vocab_size,dim])
    for word,i in word_index.items():
        vector=embedding.get(word)
        if vector is not None:
            embedding_matrix[i] = vector
    return embedding_matrix

In [ ]:
embedding_matrix = filter_embedding(embedding, tk.word_index, len(tk.word_index)+1 ,300)

In [ ]:
train_pad = all_pad[:len(train)]
test_pad = all_pad[len(train):]

In [ ]:
from keras import Sequential
from keras.layers import *

model = Sequential()
model.add(Embedding(len(tk.word_index)+1,300, input_length=52, weights=[embedding_matrix], trainable=False))
# model.add(Embedding(len(tk.word_index)+1,output_dim=10,input_length = 52)) #0이 들어가기 떄문에 1을 추가 , 중간에 들어가는 3은 단어들을 구별하는 차원 수를 나타냄
model.add(LSTM(32))
# model.add(Flatten()) #lstm이 대신 기능함 
model.add(Dense(5,activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"]) #sparse를 붙이는 이유는 분류임을 알려주기 위해
model.fit(train_pad, train["Sentiment"], epochs=5, batch_size=512)
result = model.predict(test_pad)

In [ ]:
sub = pd.read_csv("/kaggle/input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv")
sub

In [ ]:
mapping = {phrase:sentiment for _,_,phrase,sentiment in train.values}
mapping
result = result.argmax(1)
for i,j in enumerate(test["Phrase"]):
    if j in mapping:
        result[i] = mapping[j]

In [ ]:
sub["Sentiment"] = result #argmax axis 1

In [ ]:
sub.to_csv("sub_sentiment.csv",index=0)